# 원스토어 GitBook MD 파일 통합 처리

원스토어 인앱결제 API V7(SDK V21) GitBook에서 모든 링크된 MD 파일을 읽어들여 하나의 파일로 통합

In [32]:
import requests
import re
import os
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
import time
from typing import List, Dict, Set
import json

In [40]:
class GitBookMDProcessor:
    def __init__(self, base_url: str):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        self.processed_urls: Set[str] = set()
        self.md_files: Dict[str, str] = {}
        # 링크 발견 순서를 보존하기 위한 리스트
        self.ordered_urls: List[str] = []
    def fetch_page(self, url: str) -> str:
        """웹페이지를 가져옵니다."""
        try:
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            return response.text
        except Exception as e:
            print(f"❌ {url} 가져오기 실패: {e}")
            return ""
    
    def extract_md_links(self, html_content: str) -> List[str]:
        """HTML에서 MD 파일 링크를 추출합니다."""
        soup = BeautifulSoup(html_content, 'html.parser')
        md_links = []
        
        # 모든 링크 찾기
        for link in soup.find_all('a', href=True):
            href = link['href']
            
            # MD 파일 링크 패턴 확인
            if self.is_md_link(href):
                full_url = urljoin(self.base_url, href)
                md_links.append(full_url)
        
        return md_links
    
    def extract_markdown_links(self, content: str) -> List[str]:
        """마크다운 콘텐츠에서 링크 추출"""
        # [텍스트](경로) 형태의 마크다운 링크 패턴
        markdown_link_pattern = r'\[([^\]]+)\]\(([^)]+)\)'
        links = []
        
        matches = re.findall(markdown_link_pattern, content)
        for text, path in matches:
            if self.is_md_link(path):
                full_url = self.convert_to_absolute_url(path)
                if full_url:
                    links.append(full_url)
        
        return links
    
    def is_md_link(self, href: str) -> bool:
        """MD 파일 링크인지 확인합니다."""
        # GitBook MD 파일 패턴들
        md_patterns = [
            r'\.md$',  # .md로 끝나는 파일
            r'v21/[^/]+$',  # v21/ 디렉토리 아래의 파일들
            r'old-version/v16',  # 이전 버전 파일들
            r'glb',  # 글로벌 배포 가이드
        ]
        
        for pattern in md_patterns:
            if re.search(pattern, href):
                return True
        return False
    
    def convert_to_absolute_url(self, href: str) -> str:
        """상대 경로를 절대 URL로 변환"""
        # GitBook 특수 패턴 처리
        if href.startswith('(') and href.endswith(')'):
            # (v21/ov) 형태를 v21/ov로 변환
            href = href[1:-1]
        
        # .md 확장자가 없으면 추가
        if not href.endswith('.md') and not href.endswith('/'):
            href += '.md'
        
        # base_url에서 경로 추출
        base_path = '/dev/tools/billing/'
        
        # 절대 URL 생성
        if href.startswith('/'):
            absolute_url = f"https://onestore-dev.gitbook.io{href}"
        elif href.startswith('http'):
            absolute_url = href
        else:
            # 상대 경로 처리
            if href.startswith('v21/'):
                absolute_url = f"https://onestore-dev.gitbook.io{base_path}{href}"
            elif href.startswith('../'):
                # 상위 디렉토리 처리
                absolute_url = f"https://onestore-dev.gitbook.io/dev/tools/{href[3:]}"
            else:
                absolute_url = f"https://onestore-dev.gitbook.io{base_path}{href}"
        
        return absolute_url
    
    def extract_content_from_page(self, html_content: str) -> str:
        """HTML 페이지에서 실제 콘텐츠를 추출합니다."""
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # GitBook의 메인 콘텐츠 영역 찾기
        content_selectors = [
            'article',
            '.markdown',
            '.gitbook-root',
            'main',
            '.content'
        ]
        
        content = ""
        for selector in content_selectors:
            element = soup.select_one(selector)
            if element:
                # 원본 포맷팅 보존을 위한 개선된 추출
                content = self._extract_formatted_content(element)
                break
        
        # 콘텐츠를 찾지 못한 경우 전체 body 사용
        if not content:
            body = soup.find('body')
            if body:
                content = self._extract_formatted_content(body)
        
        return content
    
    def _extract_formatted_content(self, element) -> str:
        """원본 포맷팅을 보존하여 콘텐츠를 추출합니다."""
        content_parts = []
        
        # 제목 태그들 처리
        for tag in element.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
            level = int(tag.name[1])
            prefix = '#' * level
            text = tag.get_text(strip=True)
            if text:
                content_parts.append(f"{prefix} {text}")
        
        # 코드 블록 처리
        for code_block in element.find_all(['pre', 'code']):
            if code_block.name == 'pre':
                # pre 태그 내의 code 태그 찾기
                code_element = code_block.find('code')
                if code_element:
                    language = code_element.get('class', [''])[0] if code_element.get('class') else ''
                    code_content = code_element.get_text()
                    if language:
                        content_parts.append(f"```{language}")
                    else:
                        content_parts.append("```")
                    content_parts.append(code_content)
                    content_parts.append("```")
                else:
                    # pre 태그만 있는 경우
                    code_content = code_block.get_text()
                    content_parts.append("```")
                    content_parts.append(code_content)
                    content_parts.append("```")
            elif code_block.name == 'code' and not code_block.find_parent('pre'):
                # 인라인 코드
                code_content = code_block.get_text()
                content_parts.append(f"`{code_content}`")
        
        # 리스트 처리
        for ul in element.find_all('ul'):
            for li in ul.find_all('li', recursive=False):
                text = li.get_text(strip=True)
                if text:
                    content_parts.append(f"- {text}")
        
        for ol in element.find_all('ol'):
            for i, li in enumerate(ol.find_all('li', recursive=False), 1):
                text = li.get_text(strip=True)
                if text:
                    content_parts.append(f"{i}. {text}")
        
        # 테이블 처리
        for table in element.find_all('table'):
            rows = table.find_all('tr')
            if rows:
                # 헤더 행
                header_cells = rows[0].find_all(['th', 'td'])
                if header_cells:
                    header_text = ' | '.join(cell.get_text(strip=True) for cell in header_cells)
                    content_parts.append(header_text)
                    content_parts.append(' | '.join(['---'] * len(header_cells)))
                    
                    # 데이터 행들
                    for row in rows[1:]:
                        cells = row.find_all('td')
                        if cells:
                            row_text = ' | '.join(cell.get_text(strip=True) for cell in cells)
                            content_parts.append(row_text)
        
        # 일반 텍스트 처리 (이미 처리된 태그들 제외)
        for tag in element.find_all(['p', 'div', 'span', 'strong', 'em', 'a']):
            if not tag.find_parent(['pre', 'code', 'ul', 'ol', 'table']):
                if tag.name == 'strong':
                    text = tag.get_text(strip=True)
                    if text:
                        content_parts.append(f"**{text}**")
                elif tag.name == 'em':
                    text = tag.get_text(strip=True)
                    if text:
                        content_parts.append(f"*{text}*")
                elif tag.name == 'a':
                    text = tag.get_text(strip=True)
                    href = tag.get('href', '')
                    if text and href:
                        content_parts.append(f"[{text}]({href})")
                    elif text:
                        content_parts.append(text)
                else:
                    text = tag.get_text(strip=True)
                    if text:
                        content_parts.append(text)
        
        # 줄바꿈으로 연결
        return '\n\n'.join(content_parts)
    def process_url(self, url: str) -> None:
        """URL을 처리하고 MD 파일을 찾습니다. (링크 순서 보존)"""
        if url in self.processed_urls:
            return
        
        print(f"🔍 처리 중: {url}")
        self.processed_urls.add(url)
        # 순서 보존을 위해 리스트에 추가
        if url not in self.ordered_urls:
            self.ordered_urls.append(url)
        
        html_content = self.fetch_page(url)
        if not html_content:
            return
        
        # 현재 페이지의 콘텐츠 추출
        page_content = self.extract_content_from_page(html_content)
        if page_content:
            self.md_files[url] = page_content
            print(f"✅ 콘텐츠 추출 완료: {len(page_content)} 문자")
        
        # HTML 링크 추출
        html_links = self.extract_md_links(html_content)
        print(f"📎 발견된 HTML 링크: {len(html_links)}개")
        
        # 마크다운 링크 추출 (HTML 콘텐츠에서)
        markdown_links = self.extract_markdown_links(html_content)
        print(f"📎 발견된 마크다운 링크: {len(markdown_links)}개")
        
        # 모든 링크 합치기 (중복 제거하되 순서 보존)
        all_links = []
        seen_links = set()
        
        # HTML 링크 먼저 추가
        for link in html_links:
            if link not in seen_links:
                all_links.append(link)
                seen_links.add(link)
        
        # 마크다운 링크 추가
        for link in markdown_links:
            if link not in seen_links:
                all_links.append(link)
                seen_links.add(link)
        
        # 링크 발견 순서대로 처리
        for link in all_links:
            if link not in self.processed_urls:
                time.sleep(1)  # 요청 간격 조절
                self.process_url(link)  # 재귀 호출
    
    def process_all_urls_ordered(self, start_url: str) -> None:
        """링크 발견 순서를 보존하여 모든 URL을 처리합니다."""
        print(f"🚀 링크 순서 보존 방식으로 URL 처리 시작: {start_url}")
        print("=" * 60)
        
        # 초기화
        self.processed_urls.clear()
        self.md_files.clear()
        self.ordered_urls.clear()
        
        # 순서 보존 처리 시작
        self.process_url(start_url)
        
        print(f"\n📊 순서 보존 처리 완료 통계:")
        print(f"- 처리된 URL 수: {len(self.processed_urls)}")
        print(f"- 추출된 MD 파일 수: {len(self.md_files)}")
        
        # 링크 발견 순서대로 URL 출력
        print(f"📋 처리된 URL 목록 (링크 발견 순서):")
        for i, url in enumerate(self.ordered_urls, 1):
            print(f"  {i}. {url}")
    def process_all_urls_dfs(self, start_url: str) -> None:
        """DFS 방식으로 모든 URL을 순차적으로 처리합니다."""
        print(f"🚀 DFS 방식으로 URL 처리 시작: {start_url}")
        print("=" * 60)
        
        # 초기화
        self.processed_urls.clear()
        self.md_files.clear()
        
        # DFS 시작
        self.process_url(start_url)
        
        print(f"\n📊 DFS 처리 완료 통계:")
        print(f"- 처리된 URL 수: {len(self.processed_urls)}")
        print(f"- 추출된 MD 파일 수: {len(self.md_files)}")
        
        # 처리된 URL들을 순서대로 출력
        print(f"📋 처리된 URL 목록 (DFS 순서):")
        for i, url in enumerate(self.processed_urls, 1):
            print(f"  {i}. {url}")
    
    def save_combined_file(self, output_path: str) -> None:
        """ordered_urls 순서대로 통합된 파일을 저장합니다."""
        # ordered_urls 순서대로 통합 콘텐츠 생성
        combined_content = "# 원스토어 인앱결제 API V7(SDK V21) 통합 문서\n\n"
        combined_content += f"이 문서는 {len([url for url in self.ordered_urls if url in self.md_files])}개의 페이지에서 추출된 콘텐츠를 통합한 것입니다.\n\n"
        combined_content += "---\n\n"
        
        # ordered_urls 순서대로 처리
        processed_count = 0
        
        for i, url in enumerate(self.ordered_urls, 1):
            if url in self.md_files:
                processed_count += 1
                content = self.md_files[url]
                
                # URL에서 파일명 추출
                filename = url.split("/")[-1] or "index"
                if not filename.endswith(".md"):
                    filename += ".md"
                
                # 섹션 헤더 추가
                combined_content += f"## {processed_count}. {filename}\n\n"
                combined_content += f"**원본 URL:** {url}\n\n"
                combined_content += "---\n\n"
                
                # 원본 콘텐츠 그대로 추가 (형식 보존)
                combined_content += content
                combined_content += "\n\n---\n\n"
                
                print(f"✅ {processed_count}. {filename} 추가 완료 ({len(content)} 문자)")
        
        print(f"📊 총 {processed_count}개 파일 통합 완료")
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # 원본 형식 보존하여 저장
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(combined_content)
        
        print(f"✅ 통합 파일 저장 완료: {output_path}")
        print(f"📊 총 {processed_count}개 페이지 처리")
        print(f"📝 총 {len(combined_content)} 문자")
        
    def save_combined_file2(self, output_path: str) -> None:
        for i, url in enumerate(self.ordered_urls, 1):
            print(f"{i}. {url}")
            
    def save_metadata(self, output_path: str) -> None:
        """처리된 URL들의 메타데이터를 저장합니다. (순서 보존)"""
        metadata = {
            'base_url': self.base_url,
            'processed_urls': list(self.processed_urls),
            'ordered_urls': self.ordered_urls,  # 링크 발견 순서
            'md_files_count': len(self.md_files),
            'md_files': list(self.md_files.keys())
        }
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        
        print(f"✅ 메타데이터 저장 완료: {output_path}")
        print(f"📋 링크 발견 순서: {len(self.ordered_urls)}개 URL")


In [41]:
# 원스토어 GitBook MD 파일 처리 시작
base_url = "https://onestore-dev.gitbook.io/dev/tools/billing/v21.md"
processor = GitBookMDProcessor(base_url)

print(f"🚀 원스토어 GitBook MD 파일 처리 시작")
print(f"📍 대상 URL: {base_url}")
print("=" * 60)

🚀 원스토어 GitBook MD 파일 처리 시작
📍 대상 URL: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md


In [42]:
# 메인 페이지 처리
# DFS 방식으로 메인 페이지 처리
# 링크 순서 보존 방식으로 메인 페이지 처리
processor.process_all_urls_ordered(base_url)

print(f"\\n📊 처리 완료 통계:")
print(f"- 처리된 URL 수: {len(processor.processed_urls)}")
print(f"- 추출된 MD 파일 수: {len(processor.md_files)}")

# 처리된 URL들 출력
print(f"\\n📋 처리된 URL 목록:")
for i, url in enumerate(processor.processed_urls, 1):
    print(f"  {i}. {url}")

🚀 링크 순서 보존 방식으로 URL 처리 시작: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
🔍 처리 중: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
📎 발견된 HTML 링크: 0개
📎 발견된 마크다운 링크: 17개
🔍 처리 중: https://onestore-dev.gitbook.io/dev/tools/billing/old-version/v16.md
📎 발견된 HTML 링크: 0개
📎 발견된 마크다운 링크: 0개
🔍 처리 중: https://onestore-dev.gitbook.io/dev/tools/glb.md
📎 발견된 HTML 링크: 0개
📎 발견된 마크다운 링크: 1개
🔍 처리 중: https://onestore-dev.gitbook.io/dev/tools/billing/billing/v21/sdk.md
❌ https://onestore-dev.gitbook.io/dev/tools/billing/billing/v21/sdk.md 가져오기 실패: 404 Client Error: Not Found for url: https://onestore-dev.gitbook.io/dev/tools/billing/billing/v21/sdk.md
🔍 처리 중: https://onestore-dev.gitbook.io/dev/tools/billing/v21/ov.md
📎 발견된 HTML 링크: 0개
📎 발견된 마크다운 링크: 0개
🔍 처리 중: https://onestore-dev.gitbook.io/dev/tools/billing/v21/pre.md
📎 발견된 HTML 링크: 0개
📎 발견된 마크다운 링크: 0개
🔍 처리 중: https://onestore-dev.gitbook.io/dev/tools/billing/v21/test.md
📎 발견된 HTML 링크: 0개
📎 발견된 마크다운 링크: 0개
🔍 처리 중: https://onestore-dev.g

/var/folders/lr/mv7qs1n11s58s4mq9t7vlxr40000gn/T/ipykernel_96295/1113927804.py:101: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')
/var/folders/lr/mv7qs1n11s58s4mq9t7vlxr40000gn/T/ipykernel_96295/1113927804.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')


In [43]:
# 통합 파일 저장
output_file = "data/iap_gitbook5.md"
metadata_file = "data/iap_gitbook_metadata5.json"

processor.save_combined_file2(output_file)
# processor.save_metadata(metadata_file)

print(f"\\n🎉 모든 처리가 완료되었습니다!")
print(f"📁 통합 파일: {output_file}")
print(f"📁 메타데이터: {metadata_file}")

1. https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
2. https://onestore-dev.gitbook.io/dev/tools/billing/old-version/v16.md
3. https://onestore-dev.gitbook.io/dev/tools/glb.md
4. https://onestore-dev.gitbook.io/dev/tools/billing/billing/v21/sdk.md
5. https://onestore-dev.gitbook.io/dev/tools/billing/v21/ov.md
6. https://onestore-dev.gitbook.io/dev/tools/billing/v21/pre.md
7. https://onestore-dev.gitbook.io/dev/tools/billing/v21/test.md
8. https://onestore-dev.gitbook.io/dev/tools/billing/v21/sdk.md
9. https://onestore-dev.gitbook.io/dev/tools/billing/v21/references.md
10. https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations.md
11. https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.connectionstate.md
12. https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.featuretype.md
13. https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.producttype

In [7]:
# 결과 확인
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"📄 통합 파일 내용 미리보기:")
    print("=" * 60)
    print(content[:1000] + "..." if len(content) > 1000 else content)
    print("=" * 60)
    print(f"📊 파일 크기: {len(content)} 문자")
    
    # 키워드 검색
    keywords = ["launchPurchaseFlow", "PurchaseClient", "IapResult", "API", "SDK"]
    print(f"\\n🔍 키워드 검색 결과:")
    for keyword in keywords:
        count = content.lower().count(keyword.lower())
        print(f"  - {keyword}: {count}회 발견")
else:
    print("❌ 통합 파일이 생성되지 않았습니다.")

In [8]:
# 추가 처리: 특정 섹션만 추출
def extract_sections_by_keyword(content: str, keywords: List[str]) -> Dict[str, str]:
    """키워드별로 관련 섹션을 추출합니다."""
    sections = {}
    lines = content.split('\\n')
    
    current_section = ""
    current_keyword = ""
    
    for line in lines:
        # 키워드가 포함된 라인 찾기
        for keyword in keywords:
            if keyword.lower() in line.lower():
                if current_section:
                    sections[current_keyword] = current_section.strip()
                
                current_keyword = keyword
                current_section = line + "\\n"
                break
        else:
            if current_section:
                current_section += line + "\\n"
    
    # 마지막 섹션 추가
    if current_section:
        sections[current_keyword] = current_section.strip()
    
    return sections

# 주요 키워드별 섹션 추출
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    important_keywords = [
        "launchPurchaseFlow",
        "PurchaseClient",
        "IapResult",
        "API V7",
        "SDK V21",
        "PNS",
        "정기 결제"
    ]
    
    sections = extract_sections_by_keyword(content, important_keywords)
    
    print(f"📑 키워드별 섹션 추출 결과:")
    for keyword, section in sections.items():
        print(f"\\n🔍 {keyword}:")
        print(f"   길이: {len(section)} 문자")
        print(f"   미리보기: {section[:200]}...")
        print("-" * 50)

In [26]:
# ordered_urls 순서대로 통합 파일 생성
output_file_ordered = "data/iap_gitbook_ordered.md"
metadata_file_ordered = "data/iap_gitbook_metadata_ordered.json"

# ordered_urls 순서대로 통합 파일 저장
processor.save_combined_file(output_file_ordered)
processor.save_metadata(metadata_file_ordered)

print(f"\\n🎉 ordered_urls 순서 기반 통합 완료!")
print(f"📁 통합 파일: {output_file_ordered}")
print(f"📁 메타데이터: {metadata_file_ordered}")

# 결과 확인
if os.path.exists(output_file_ordered):
    with open(output_file_ordered, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"\\n📄 ordered_urls 순서 통합 파일 내용 미리보기:")
    print("=" * 60)
    print(content[:1000] + "..." if len(content) > 1000 else content)
    print("=" * 60)
    print(f"📊 파일 크기: {len(content)} 문자")
    
    # ordered_urls에서 실제 처리된 파일 수 확인
    processed_urls = [url for url in processor.ordered_urls if url in processor.md_files]
    print(f"\\n📋 ordered_urls 순서 처리 결과:")
    print(f"- 총 ordered_urls: {len(processor.ordered_urls)}개")
    print(f"- 실제 처리된 파일: {len(processed_urls)}개")
    print(f"- 처리된 파일 목록:")
    for i, url in enumerate(processed_urls, 1):
        filename = url.split('/')[-1] or 'index'
        if not filename.endswith('.md'):
            filename += '.md'
        print(f"  {i}. {filename}")
else:
    print("❌ ordered_urls 순서 통합 파일이 생성되지 않았습니다.")

✅ 통합 파일 저장 완료: data/iap_gitbook_ordered.md
📊 총 5개 페이지 처리
📝 총 264257 문자
✅ 메타데이터 저장 완료: data/iap_gitbook_metadata_ordered.json
📋 링크 발견 순서: 73개 URL
\n🎉 ordered_urls 순서 기반 통합 완료!
📁 통합 파일: data/iap_gitbook_ordered.md
📁 메타데이터: data/iap_gitbook_metadata_ordered.json
\n📄 ordered_urls 순서 통합 파일 내용 미리보기:
# 원스토어 인앱결제 API V7(SDK V21) 통합 문서

이 문서는 5개의 페이지에서 추출된 콘텐츠를 통합한 것입니다.

---

## 1. flutter-sdk-v21#undefined-9.md

**원본 URL:** https://onestore-devs-organization.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-9.md

---

#Flutter에서 원스토어 인앱 결제 사용하기 (SDK v21)

## 개요

## 개발 버전

## 플러그인 설정

### pubspec.yaml 파일에 플러그인 추가하기

### Android build.gradle에 종속성 추가하기

### AndroidMainifest.xml 파일에 <queries> 추가하기

### 글로벌 스토어 배포를 위한 테스트 옵션 설정

## 앱에서 원스토어 인앱 결제 라이브러리 적용하기

### 로그 레벨 설정

### 로그인 요청하기

### 구매 라이브러리 초기화

### 구매 데이터 업데이트 및 오류 응답 청취하기

### 상품 상세 정보 조회하기

### 구매 요청하기

### 정기 결제 업그레이드 또는 다운그레이드

### 구매 후 처리

#### 소비하기 (Consume)

#### 확인하기 (Acknowledge)

### 구매 내역 조회하기

### 정기 결제 관리 

In [9]:
# RAG용 데이터 준비
def prepare_rag_data(content: str, output_dir: str = "data/iap_gitbook_sections"):
    """RAG용으로 섹션별로 파일을 분할하여 저장합니다."""
    os.makedirs(output_dir, exist_ok=True)
    
    # 섹션별로 분할
    sections = content.split('## ')
    
    for i, section in enumerate(sections[1:], 1):  # 첫 번째는 제목이므로 제외
        lines = section.split('\\n')
        if lines:
            title = lines[0].strip()
            content = '\\n'.join(lines[1:]).strip()
            
            # 파일명 생성
            filename = f"section_{i:03d}_{title.replace(' ', '_').replace('/', '_')[:50]}.md"
            filepath = os.path.join(output_dir, filename)
            
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(f"# {title}\\n\\n{content}")
            
            print(f"✅ {filename} 저장 완료 ({len(content)} 문자)")
    
    print(f"\\n📁 총 {len(sections)-1}개 섹션 파일이 {output_dir}에 저장되었습니다.")

# RAG용 데이터 준비
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    prepare_rag_data(content)
else:
    print("❌ 통합 파일이 없어 RAG 데이터를 준비할 수 없습니다.")